In [1]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go

from datetime import datetime

### 1 - Import test data

In [2]:
dataF = yf.download("EURUSD=X", start="2024-10-03", end="2024-11-29", interval='15m')
dataF.iloc[:,:]
#dataF.Open.iloc

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X
Datetime,,,,,,
2024-10-03 00:00:00,1.104850,1.104850,1.104972,1.104728,1.104972,0
2024-10-03 00:15:00,1.104972,1.104972,1.104972,1.104850,1.104850,0
2024-10-03 00:30:00,1.104972,1.104972,1.105094,1.104850,1.104972,0
2024-10-03 00:45:00,1.104850,1.104850,1.105094,1.104850,1.104972,0
2024-10-03 01:00:00,1.104850,1.104850,1.104972,1.104728,1.104850,0
...,...,...,...,...,...,...
2024-11-28 22:45:00,1.055632,1.055632,1.055966,1.055632,1.055855,0
2024-11-28 23:00:00,1.056078,1.056078,1.056078,1.055632,1.055743,0


In [3]:
dataF.columns = dataF.columns.droplevel(1)

In [4]:
dataF.reset_index(inplace=True)

In [5]:
# Calculate the 50-day EMA
dataF['EMA_50'] = dataF['Adj Close'].ewm(span=50, adjust=False).mean()

In [6]:
fig = go.Figure(data=[go.Candlestick(x= dataF.index, # dataF["Datetime"]
                open=dataF['Open'],
                high=dataF['High'],
                low=dataF['Low'],
                close=dataF['Close'])])

fig.add_trace(go.Scatter(
    x= dataF.index, # dataF["Datetime"]
    y=dataF['EMA_50'],
    mode='lines',
    name='50-day EMA',
    line=dict(color='blue', width=2)
))

fig.update_layout(height=800)

fig.show()

In [7]:
print(dataF["EMA_50"].iloc[-1])

1.055576706316363


### 2 - Define your signal function

#### We define the buying strategy

In [8]:
# Note that in between all these functions time can flow (for example between buy and get_swing_high_point_before_pullback) or no time can flow,
# i.e. the functions are executed in the same time frame (for example between pullback and get_swing_high_point_before_pullback there is no time 
# in between, the functions are executed one after the other).

In [9]:
def price_below_EMA(df):
    # Is True if price is below 50 day EMA 
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    EMA_50 = df["EMA_50"].iloc[-1]

    if open >= EMA_50 and close <= EMA_50:
        return True
    
    return False

def strategy_start(df):
    # Strategy starts once the price is above EMA and a candle closes above it
    close = df.Close.iloc[-1]
    EMA_50 = df["EMA_50"].iloc[-1]

    if close >= EMA_50:
        return True
    
    return False

def pullback(df):
    # We consider a Pullback when we have at least 2 opposite candles coming down
    bool1 = (df.Open.iloc[-1] - df.Close.iloc[-1] >= 0)
    bool2 = (df.Open.iloc[-2] - df.Close.iloc[-2] >= 0)

    if bool1 and bool2:
        return True
    
    return False

def get_swing_high_point_before_pullback(df):
    # We apply this function only if the other one is True

    # Gets high point before pullback
    high_point = df.High.iloc[-3]

    len_high_point_candle = df.Close.iloc[-3] - df.Open.iloc[-3]

    return high_point, len_high_point_candle

def invalid_trade_1(df):
    # Check this condition before buying

    # If the prices closes below the EMA after the pullback then the trade is invalid
    if df.Close.iloc[-1] < df["EMA_50"].iloc[-1]:
        # Trade is invalid
        return True
    # trade is valid
    return False

def invalid_trade_2(df, len_high_point_candle):
    # This functions needs to be called right after get_swing_high_point_before_pullback
    # Calculate tthe mean height of the last 40 candles

    # If the breakcoutcandle is 3 or 4 times bigger than the candles before, then the trade is invalid
    df['AbsDiff'] = (df.Close - df.Open).abs()
    mean_abs_diff = df['AbsDiff'].iloc[:50].mean()

    if len_high_point_candle >= 3*mean_abs_diff:
        return True
    return False


def buy(df, high_point):
    # The body of the candle needs to close above the swing high point
    close = df.Close.iloc[-1]

    if close >= high_point:
        return True
    
    return False

In [10]:
# The bots need to be able to do 2 things, see future trading strateegies, and if it is started in one it should be able to work from there
# For this second option I need to go row by row and append to some available data and then run the function below on these partial dataframes
# I will also do this for the testing now but it has to be done once I am connected to the broker also

# I need to implement take proft target

In [22]:
# Do on running data
def trading_strategy(df):
    cond_1 = False
    cond_2 = False
    cond_3 = False

    if price_below_EMA(df):
        print("test1")
        cond_1 = True
    if cond_1 & strategy_start(df):
        print("test2")
        cond_2 = True
    if cond_1 & cond_2 & pullback(df):
        cond_3 = True

    if cond_3:
        high_point, len_high_point_candle = get_swing_high_point_before_pullback(df)

    if cond_3 and buy(df, high_point):
        print("BUY")
        return "buy"

    if cond_3 and invalid_trade_1(df):
        print("Invalid trade")
        return "invalid"
    
    if cond_3 and invalid_trade_2(df, len_high_point_candle):
        print("Invalid trade")
        return "invalid"

In [12]:
test = dataF[dataF["Datetime"] >= "2024-11-19"]

In [13]:
print(test.dtypes)

Price
Datetime     datetime64[ns]
Adj Close           float64
Close               float64
High                float64
Low                 float64
Open                float64
Volume                int64
EMA_50              float64
dtype: object


In [23]:
available_df = pd.DataFrame(columns=test.columns)

In [24]:
for index, row in test.iterrows():
    row_df = pd.DataFrame([row])
    available_df = pd.concat([available_df, row_df])
    if len(available_df) >= 3:
        trading_strategy(available_df)

/var/folders/6b/r3xhr4d94wn52lq9yxzbnnzm0000gn/T/ipykernel_47212/1574225811.py:3: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



test1
test1
test1
test1
test1
test1
test1
test1
test1
test1
test1
test1
test1
test1
test1
test1
test1
test1
test1
test1
test1
test1
test1
test1
test1
test1
test1
test1
test1
test1
test1
test1
test1
test1


In [ ]:
display(available_df)

### Rest of code, not done yet

In [ ]:
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]
    
    # Bearish Pattern
    if (open>close and 
    previous_open<previous_close and 
    close<previous_open and
    open>=previous_close):
        return 1

    # Bullish Pattern
    elif (open<close and 
        previous_open>previous_close and 
        close>previous_open and
        open<=previous_close):
        return 2
    
    # No clear pattern
    else:
        return 0

signal = []
signal.append(0)
for i in range(1,len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))
#signal_generator(data)
dataF["signal"] = signal

In [ ]:
dataF.signal.value_counts()
#dataF.iloc[:, :]

### 3 - Connect to the market and execute trades

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [ ]:
from config import access_token, accountID
def get_candles(n):
    #access_token='XXXXXXX'#you need token here generated from OANDA account
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)


In [ ]:
def trading_job():
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])
    
    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:])#
    
    # EXECUTING ORDERS
    #accountID = "XXXXXXX" #your account ID here
    client = API(access_token)
         
    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])
    
    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio
    
    print(dfstream.iloc[:-1,:])
    print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    signal = 2
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

### 4 - Executing orders automatically with a scheduler

In [ ]:
trading_job()

#scheduler = BlockingScheduler()
#scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2022-01-12 12:00:00', timezone='America/Chicago')
#scheduler.start()